# Parallel Scenario Calculation with DiffFusion.jl

In this notebook, we present scaling results from parallel calculation of Monte Carlo valuation scenarios. With these results, we want to discuss the following questions:

  1. How well can we exploit parallel computation capabilities with the [DiffFusion.jl](https://github.com/frame-consulting/DiffFusion.jl) framework?

  2. What do we need to keep in mind when using Julia's [parallel computing](https://docs.julialang.org/en/v1/manual/parallel-computing/) functionalities?

Scenarios are calculated for random but representative portfolios of [interest rate swaps](https://en.wikipedia.org/wiki/Interest_rate_swap). Business use case is, for example, exposure simulation for [counterparty credit risk](https://en.wikipedia.org/wiki/Credit_risk#Counterparty_risk) modelling. 

As parallelisation techniques we consider multi-threading, multi-processing and a combination of both approaches.

Results are calculated on an hpc6a AWS EC2 instance with Julia 1.10.7 and DiffFusion.jl 0.7.0.

## Packages and Prerequisites

In [ ]:
using Pkg
Pkg.activate("../.")

In [ ]:
using CSV
using DataFrames
using DiffFusion
using OrderedCollections
using Plots
using Printf
using YAML

## Background Exposure Simulation

The business background of our example workload is exposure simulation for counterparty credit risk. From a computational perspective, exposure simulation can be represented as a sequence of calculation steps:

 - Simulate key financial risk factors (interest rates, exchange rates, stock prices) of an economy for a grid of future simulation/observation times and a number of simulation scenarios (i.e. number of paths).

 - For each future time grid and simulated scenario calculate the *fair price* of a portfolio of financial instruments. This yields the set of scenario valuation results.

 - Aggregate the scenario valuation results and calculate relevant risk figures.

We illustrate the simulation in below graphs.


| ![Example path](example_path.png) | ![Exposure calculation](example_exposures.png)
|:--:|:--:|
| *Example scenarios on a single path.* | *Exposure with and without collateral.* |

(Below calculation cells are commented out because large data files are not included in repository.)

In [ ]:
# scens_w_cb = DiffFusion.deserialise(YAML.load_file("data/scens_w_cb.yml"; dicttype=OrderedDict{String,Any}))
# scens_cva = DiffFusion.deserialise(YAML.load_file("data/scens_cva.yml"; dicttype=OrderedDict{String,Any}))

In [ ]:
# scens_agg = DiffFusion.aggregate(scens_w_cb, false, true)
# scens_ee = DiffFusion.expected_exposure(scens_w_cb, true, true, false)
# scens_ee_w_cb = DiffFusion.expected_exposure(scens_w_cb, false, true, true);

In [ ]:
#=

x = range(start=0, stop=10, step=1)
xticks_ = (x, [string(Int(t)) for t in x])
path_idx = 1024
scaling = 1.e6

plot( # base plot
    xticks = xticks_,
    # xaxis = :log,
    annotationfontsize = 8,
    xlims = (0, 10),
    # ylims = (0, 100),
    xlabel = "observation time (years)",
    ylabel = "scenario value (Mio USD)",
    size = (800, 600) .* 0.7,
    labelfontsize = 8,
    title = "Portfolio valuation, 128 instruments",
    titlefontsize = 10,
)

plot!(
    scens_w_cb.times, scens_w_cb.X[path_idx,:,1] / scaling,
    label="Swap portfolio (path #$path_idx)",
    seriestype=:path,
    linestyle=:solid,
    lw = 1,
    seriescolor = :green,
)
plot!(
    scens_w_cb.times, scens_w_cb.X[path_idx,:,2] / scaling,
    label="Collateral balance",
    seriestype=:path,
    linestyle=:solid,
    lw = 1,
    seriescolor = :blue,
)
plot!(
    scens_agg.times, scens_agg.X[path_idx,:,1] / scaling,
    label="Collateralised portfolio",
    seriestype=:path,
    linestyle=:solid,
    lw = 1,
    seriescolor = :red,
)

=#


In [ ]:
# png("example_path.png")

In [ ]:
#=

plot( # base plot
    xticks = xticks_,
    # xaxis = :log,
    annotationfontsize = 8,
    xlims = (0, 10),
    ylims = (0, 100),
    xlabel = "observation time  (years)",
    ylabel = "scenario value (Mio USD)",
    size = (800, 600) .* 0.7,
    labelfontsize = 8,
    title = "Expected exposure, 128 instruments",
    titlefontsize = 10,
)
plot!(
    scens_ee.times, scens_ee.X[1,:,1] / scaling,
    label="Uncollateralised portfolio",
    seriestype=:path,
    linestyle=:solid,
    lw = 1,
    seriescolor = :green,
)
plot!(
    scens_ee_w_cb.times, scens_ee_w_cb.X[1,:,1] / scaling,
    label="Collateralised portfolio",
    seriestype=:path,
    linestyle=:solid,
    lw = 1,
    seriescolor = :red,
)

=#

In [ ]:
# png("example_exposures.png")

In [ ]:
#=

plot( # base plot
    xticks = xticks_,
    # xaxis = :log,
    annotationfontsize = 8,
    xlims = (0, 10),
    # ylims = (0, 100),
    xlabel = "observation time",
    ylabel = "scenario value (1k USD)",
    size = (800, 600) .* 0.7,
    labelfontsize = 8,
    title = "Credit Valuation Adjustment per year",
    titlefontsize = 10,
)
plot!(
    scens_cva.times, scens_cva.X[1,:,1] / scaling * 1_000 / 0.25,
    label="Collateralised portfolio",
    seriestype=:path,
    linestyle=:solid,
    lw = 1,
    seriescolor = :red,
)

=#

## Parallelisation of Scenario Valuation

Exposure simulation calculations typically scale with the number of modelled financial risk factors, the number of simulation times and the number of simulation scenarios.

Most critically from a computational perspective is the second step of scenario valuation. The computational effort also scales with the number of instruments in the portfolio. And the individual calculations are typically more complex than the calculations from the first and second step.

Fortunately, scenario valuation is (in most cases) independent with respect to financial product and simulation time. This opens up the possibility of parallelisation.


## Run Time Results

In [ ]:
table_mt = DataFrame(CSV.File("data/Workloads.MT.csv"))
table_mp = DataFrame(CSV.File("data/Workloads.MP.csv"))
table_mx = DataFrame(CSV.File("data/Workloads.MX.csv"));

In [ ]:
function plot_run_times(n_prods, ylims)
    idx_mt = (table_mt.n_prods .== n_prods)
    idx_mp = (table_mp.n_prods .== n_prods)
    idx_mx = (table_mx.n_prods .== n_prods)
    #
    y_mt = table_mt[idx_mt, "run_time"]
    y_mp = table_mp[idx_mp, "run_time"]
    y_mx = table_mx[idx_mx, "run_time"]
    #
    x = [ 6, 12, 24, 48, 96 ]
    xticks = (x, [string(Int(t)) for t in x])
    #
    plot( # base plot
        xticks = xticks,
        xaxis = :log,
        annotationfontsize = 8,
        xlims = (5, 128),
        ylims = ylims,
        xlabel = "threads × processes",
        ylabel = "run time (sec.)",
        size = (800, 600) .* 0.7,
        labelfontsize = 8,
        title = "DiffFusion.jl workload, hpc6a.48xlarge, $n_prods instruments",
        titlefontsize = 10,
    )
    #
    plot!(
        x, y_mt,
        label="multi-threading",
        seriestype=:path,
        linestyle=:solid,
        lw = 1,
        seriescolor = :green,
        marker = :rect,
        markersize = 4,
        markercolor = :green,
        markerstrokecolor = :green,    
    )
    annotate!(
        x * 1.2,
        y_mt + [ 0, 0, 0, 0, 0 ],
        [ @sprintf("%.1f", y) for y in y_mt ]
    )
    #
    plot!(
        x, y_mp,
        label="multi-processing",
        seriestype=:path,
        linestyle=:solid,
        lw = 1,
        seriescolor = :blue,
        marker = :rect,
        markersize = 4,
        markercolor = :blue,
        markerstrokecolor = :blue,
    )
    annotate!(
        x * 1.2,
        y_mp + [ 0, 0, 0, 0, 0 ],
        [ @sprintf("%.1f", y) for y in y_mp ]
    )
    #
    plot!(
        x, y_mx,
        label="multi-processing w/ 6 threads",
        seriestype=:path,
        linestyle=:solid,
        lw = 1,
        seriescolor = :red,
        marker = :rect,
        markersize = 4,
        markercolor = :red,
        markerstrokecolor = :red,
    )
    annotate!(
        x * 1.2,
        y_mx + [ 0, 0, 0, 0, 0 ],
        [ @sprintf("%.1f", y) for y in y_mx ]
    )    
end

In [ ]:
plot_run_times(256, (0, 20))

In [ ]:
# png("run_times_256.png")

In [ ]:
plot_run_times(512, (0, 40))

In [ ]:
# png("run_times_512.png")

In [ ]:
plot_run_times(1024, (0, 70))

In [ ]:
# png("run_times_1024.png")